<a href="https://colab.research.google.com/github/esuda/mineiracao_dados_complexos/blob/master/Big%20Data/Big_Data_010_Trabalho_1_Auxilio_Emergencial_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pacote de dados


*   Abra o link do pacote de dados: https://tinyurl.com/bd10-auxilio
*   Selecione a opção "Adicionar Atalho ao Drive"


In [1]:
!pip install mrjob

     |████████████████████████████████| 439 kB 5.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive/auxilio/"

 auxilio_0.1pct.csv.gz
 auxilio_10pct.csv.gz
 auxilio_1pct.csv.gz
 auxilio.csv
 auxilio.csv.gz
'Dicionário de Dados - Auxílio Emergencial - Portal da transparência.pdf'
 estimativa_dou_2021.csv


In [14]:
!zcat "/content/drive/My Drive/auxilio/auxilio.csv.gz" | head

"MÊS DISPONIBILIZAÇÃO";"UF";"CÓDIGO MUNICÍPIO IBGE";"NOME MUNICÍPIO";"NIS BENEFICIÁRIO";"CPF BENEFICIÁRIO";"NOME BENEFICIÁRIO";"NIS RESPONSÁVEL";"CPF RESPONSÁVEL";"NOME RESPONSÁVEL";"ENQUADRAMENTO";"PARCELA";"OBSERVAÇÃO";"VALOR BENEFÍCIO"
"202011";"";"";"";"00000000000";"***.643.870-**";"";"-2";"";"Não se aplica";"EXTRA CADUN";"7ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.108.816-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"6ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.095.215-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"6ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.108.816-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"7ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"5ª";"Não há";"600,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"3ª";"Não há";"600,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Nã

In [ ]:
a = '202011;BA;;QUIXABEIRA;00000000000;***.560.975-**;JOSE NAILTON CUNHA DE OLIVEIRA;-2;;Não se aplica;EXTRA CADUN;7ª;Não há;300,00'

a = re.sub("\"", "", a)
first, second, third, *other, last = a.split(';')
print(last, second, third)


300,00 BA 


In [ ]:
import re
print(last)
try:
  print(int(float(re.sub("[,]",".",re.sub("[.]", "",last)))))
except:
  print("falhou")

300,00
300


In [ ]:
!cat "/content/drive/My Drive/auxilio/estimativa_dou_2021.csv" | grep "52,18391"

GO,52,18391,Professor Jamil,3211


In [ ]:
popFile = open("/content/drive/My Drive/auxilio/estimativa_dou_2021.csv", "r")
popDict = {}
for line in popFile :
  try :
    uf,coduf,codmun,nome,pop = line.split(',')
    popDict[coduf+codmun] = [nome, int(pop)]
  except:
    pass      
popFile.close()

In [ ]:
type(popDict["5218391"][1])

int

In [5]:
%%file auxilio.py
from mrjob.job import MRJob
from mrjob.protocol import TextProtocol
import re
import logging

class Auxilio(MRJob):

  OUTPUT_PROTOCOL = TextProtocol

  def configure_args(self):
    super(Auxilio, self).configure_args()
    self.add_passthru_arg('--popFile', default="/content/drive/My Drive/auxilio/estimativa_dou_2021.csv")

  def mapper(self, _, line):
    try :
      # modificar para extrair os dados da linha e gerar saída conforme enunciado
      
      #---------------
      # Inicio Solucao
      #---------------
      #logging.warning("inicio: "+line)

      #Retira identificador de texto de aspas duplas
      line = re.sub("[\"]", "", line)
      #logging.warning("substitui aspas: \n"+line)

      #Retira tudo o que nao for letra, espaco e ponto e virgula
      #line = re.sub("[^a-z0-9\s;.]", "", line.lower())

      #Desempacota as variaveis
      pay_date, uf, cod_ibge, *other, pay_val = line.split(";")
      #logging.warning("codibge e valor: "+cod_ibge+"\t"+pay_val)

      if cod_ibge == '':
        raise ValueError("Codigo IBGE vazio")
      elif pay_val == '':
        raise ValueError("Valor pago vazio")

      #Transforma o valor pago em float
      pay_val = float(re.sub("[,]",".",re.sub("[.]", "", pay_val)))
      #logging.warning("codibge e valor transformado: "+cod_ibge+"\t"+pay_val)

      #Retorna a chave e o valor pago
      yield (cod_ibge, pay_val)

    except ValueError as error:
      logging.warning(error.args)
      logging.warning("linha descartada: "+line)
      pass

  def reducer_init(self):
    logging.warning("criando dicionário de municipios")
    popFile = open(self.options.popFile, "r")
    self.popDict = {}
    for line in popFile :
      try :
        uf,coduf,codmun,nome,pop = line.split(',')
        self.popDict[coduf+codmun] = [nome, int(pop)]
      except:
        pass      
    popFile.close()

  def reducer(self, key, values):
    try :
      # modificar para obter município e população a partir da chave, e 
      # calcular pagamentos per capita de acordo com o total de valores
      #---------------
      # Inicio Solucao
      #---------------
      total_val = sum(values)
      city, pop = self.popDict[key]
      yield city, str(total_val/pop)
    except :
      pass

if __name__ == '__main__':
    logging.basicConfig(filename="log.txt")
    Auxilio.run()

Writing auxilio.py


In [10]:
!python auxilio.py "/content/drive/My Drive/auxilio/auxilio_10pct.csv.gz" 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/auxilio.root.20211030.212728.887854
Running step 1 of 1...
job output is in /tmp/auxilio.root.20211030.212728.887854/output
Streaming final output from /tmp/auxilio.root.20211030.212728.887854/output...
Castelo	14.244783433449257
Colatina	13.984326094477925
Conceição da Barra	12.655357539947266
Conceição do Castelo	15.677970047334524
Divino de São Lourenço	18.85198300283286
Domingos Martins	17.845222743259086
Dores do Rio Preto	17.132342116885027
Ecoporanga	15.909618427993669
Fundão	12.452656508333705
Governador Lindenberg	13.005671801946807
Guaçuí	13.722140762463344
Guarapari	17.08875210110191
Ibatiba	18.05205141618713
Ibiraçu	12.142744665774348
Ibitirama	18.33850509626274
Iconha	13.72633671802883
Irupi	17.96035693387946
Itaguaçu	14.411171506222285
Itapemirim	18.685899819778584
Itarana	16.059810217578836
Iúna	17.10827072781045
Jaguaré	14.524581341606256
Jerônimo Mon

In [11]:
!cat log.txt

INFO:mrjob.conf:No configs found; falling back on auto-configuration
INFO:mrjob.runner:Creating temp directory /tmp/auxilio.root.20211030.212728.887854
INFO:mrjob.sim:Running step 1 of 1...
INFO:mrjob.runner:job output is in /tmp/auxilio.root.20211030.212728.887854/output
INFO:mrjob.runner:Streaming final output from /tmp/auxilio.root.20211030.212728.887854/output...
INFO:mrjob.runner:Removing temp directory /tmp/auxilio.root.20211030.212728.887854...


In [12]:
!rm log.txt

In [13]:
!python auxilio.py "/content/drive/My Drive/auxilio/auxilio.csv.gz" 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/auxilio.root.20211030.214249.457087
Running step 1 of 1...
Traceback (most recent call last):
  File "auxilio.py", line 82, in <module>
    Auxilio.run()
  File "/usr/local/lib/python3.7/dist-packages/mrjob/job.py", line 616, in run
  File "/usr/local/lib/python3.7/dist-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/usr/local/lib/python3.7/dist-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/usr/local/lib/python3.7/dist-packages/mrjob/runner.py", line 503, in run
    self._run()
  File "/usr/local/lib/python3.7/dist-packages/mrjob/sim.py", line 161, in _run
    self._run_step(step, step_num)
  File "/usr/local/lib/python3.7/dist-packages/mrjob/sim.py", line 170, in _run_step
    self._run_streaming_step(step, step_num)
  File "/usr/local/lib/python3.7/dist-packages/mrjob/sim.py", line 181, in _run_streaming_step
    se

In [ ]:
drive.flush_and_unmount()